In [1]:
from scipy.optimize import linprog
import numpy as np

In [59]:
value_a = np.random.rand(2)
value_b = np.random.rand(2)
p = np.random.rand(1)
print('scenario_a: {}\nscenario_b: {}\np: {}'.format(value_a, value_b, p))

value = np.concatenate([[0, 0], p * value_a, (1 - p) * value_b])

A = np.array([[1, 1, 0, 0, 0, 0],
              [1, 0, -1, 0, 0, 0],
              [0, 1, 0, -1, 0, 0],
              [1, 0, 0, 0, -1, 0],
              [0, 1, 0, 0, 0, -1]], dtype=float)
b = np.array([1, 0, 0, 0, 0], dtype=float)

print('c: {}\nA:\n{}\nb: {}'.format(c, A, b))

cost = -value
linprog(c=cost, A_eq=A, b_eq=b)

scenario_a: [0.5652663  0.46984572]
scenario_b: [0.36443691 0.99353977]
p: [0.29537932]
c: [-0.         -0.         -0.12261929 -0.27037918 -0.41354141 -0.41522923]
A:
[[ 1.  1.  0.  0.  0.  0.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.]]
b: [1. 0. 0. 0. 0.]


     fun: -0.8388513783665805
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([0., 1., 0., 1., 0., 1.])